In [1]:
import flyte
import flyte.remote as remote

import asyncio
from typing import List

In [2]:
flyte.init_from_config("../../config.yaml")

In [3]:
env = flyte.TaskEnvironment(
    name="env",
    image=flyte.Image
        .from_debian_base()
        .with_pip_packages("pandas", "requests"),
)

In [4]:
@env.task()
async def hello_worker(id: int) -> str:
    print(f"hello, my id is: {id} and I am being run by Action: {flyte.ctx().action}")
    return str(id)


In [5]:
@env.task()
async def hello_driver(ids: List[int] = [1, 2, 3]) -> List[str]:
    coros = []
    with flyte.group("fanout-group"):
        for id in ids:
            coros.append(hello_worker(id))
            print(f"working with id: {id}")

        vals = await asyncio.gather(*coros)

    return vals

In [6]:
run = await flyte.run.aio(hello_driver, ids=[1, 2, 3, 4, 5])

09:22:30.762712 WARNING  remote_builder.py:81 - [34mImage 356633062068.dkr.ecr.us-east-2.amazonaws.com/union/demo:flyte-951733db85f53c48a3610efd720ef9f9 found. 
                         Skip building.[0m

09:22:30.773632 WARNING  _deploy.py:160 - Built Image for environment env, image:                                                                               
                         356633062068.dkr.ecr.us-east-2.amazonaws.com/union/demo:flyte-951733db85f53c48a3610efd720ef9f9

In [7]:
run.url

'https://demo.hosted.unionai.cloud/v2/runs/project/flytesnacks/domain/development/r6bvjrh9g8d8bjzhhf68'

In [8]:
run.name

'r6bvjrh9g8d8bjzhhf68'

In [9]:
run.show_logs()

Output()

Output()

Scrolled 5 lines of logs.

In [10]:
run.details()

RunDetails(
    labels='',
    annotations='',
    env-vars='values {\n  key: "LOG_LEVEL"\n  value: "30"\n}\n',
    is-interruptible='',
    cache-overwrite=False,
    name='r6bvjrh9g8d8bjzhhf68',
    start_time='2025-08-20T16:22:31.728292+00:00',
    end_time=None,
    run_time='20 secs',
    phase='PHASE_RUNNING',
    error='NA',
    task='env.hello_driver',
    task_type='python',
    task_version='5db012a547f1e965b36e4d7f7121ef95',
    attempts=[],
    error='NA',
    phase='PHASE_RUNNING',
    group='',
    parent=''
)

In [11]:
actions = [a.name for a in remote.Action.listall(for_run_name=run.name)]

In [12]:
actions

['a0',
 'cxhqelhonglyw9ulk0w379wyo',
 '3j1bko7ek2e4adzyfaoyn7o9f',
 'ej2ocd01rc8kf91xjbur6mnoa',
 '6saok0wz5v9g7vauj3io9xsrw',
 '12nq9ciau4y5hohmy4jdt5dd7']

In [13]:
await remote.Action.get(run_name="r9wvncwtz2v42qs6sgv6", name="bbnt6kwprld4km8wi753mtvnu").show_logs()

Output()

Scrolled 1 lines of logs.

TypeError: object NoneType can't be used in 'await' expression

In [ ]:
for action in remote.Action.listall(for_run_name=run.name):
    await remote.Action.get(run_name="r9wvncwtz2v42qs6sgv6", name=action.name).show_logs(raw=True)